In [ ]:
from __future__ import absolute_import, division, print_function

import numpy as np
import pandas as pd
import os
import logging
import glob
import sys

from skimage import io as imio, transform as imtransform
from matplotlib import pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
from torchvision import transforms
from PIL import Image

In [ ]:
xform = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
])

In [ ]:
df_pred = pd.read_csv('/home/quoc_vuong/Downloads/resnet101_120_public.csv')
import json

with open('../../data/train_val2018.json') as f:
    data_str = json.load(f)
    df_trainval = pd.read_json(data_str, orient='records')

In [ ]:
dir_pub = '../../data/Public/'
dir_trainval = '../../data/TrainVal/'

In [ ]:
size = 8
pick = 1
per_class = 6
subdf = df_pred.sample(size)

fig, axes = plt.subplots((pick + 1) * size, per_class,
        figsize=(2.5 * per_class, 2.5 * size * (pick + 1)), squeeze=False)
fig.subplots_adjust(hspace=.01, wspace=.01)

for _ax in axes:
    for ax in _ax:
        ax.axis('off')

for i in range(subdf.shape[0]):
    r = subdf.iloc[i]
    im = Image.open(os.path.join(dir_pub, '{}.jpg'.format(r['id'])))
    ax = axes[(pick + 1) * i, 0]
    ax.imshow(np.array(xform(im)))
    ax.text(1.05, .5, '{} -- {}'.format(r['id'], r['predicted']),
        horizontalalignment='left',
        transform=ax.transAxes)
    
    klasses = [int(c) for c in r['predicted'].split(' ')]
    for j, c in enumerate(klasses[:pick]):
        subdf_trainval = df_trainval[df_trainval['category'] == c].sample(per_class)
        for k in range(subdf_trainval.shape[0]):
            r2 = subdf_trainval.iloc[k]
            im2 = Image.open(os.path.join(dir_trainval, str(c), '{}.jpg'.format(r2['id'])))
            ax = axes[(pick + 1) * i + j + 1, k]
            ax.imshow(np.array(xform(im2)))
fig.show()